In [1]:
from functools import reduce
import os
from datetime import datetime

In [2]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [3]:
from config import spark

22/04/25 17:35:24 WARN Utils: Your hostname, eniac resolves to a loopback address: 127.0.1.1; using 192.168.100.6 instead (on interface wlp3s0)
22/04/25 17:35:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/25 17:35:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/25 17:35:25 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [4]:
datalake = '/media/fabio/19940C2755DB566F/PAMepi/datalake/DATALAKE/'

In [5]:
version = '2022-04-19'

In [6]:
dfs = [
    spark.read.csv(os.path.join(datalake, version, 'SRAG', file), header=True, sep=';')
    for file in os.listdir(os.path.join(datalake, version, 'SRAG'))
]

In [7]:
df = reduce(lambda d1, d2: d1.unionByName(d2, allowMissingColumns=True), dfs)

In [8]:
df = df.withColumn('LVAC', F.lit(None))

In [9]:
coronavac_list = [
    'CORONAVAC', 'BUTANTAN',  'SINOVAC', 'IB', 'SINO', 'BUTA',
    'CORAVAC', 'CORONA', 'CORONO', 'BUATN', 'CORANA', 'BUNTAN',
    'BUTU', 'SIONOVAC', 'CORONSVAC', 'COROVACAC', 'COORNAVAC',
    'COVONAVA', 'CORFONAVAC', 'TANTAN', 'SIVOVAC', 'CORO', 'CONO',
    'BHUTANTAN', 'CORANOVAC', 'CORNAVAC', 'CINOFARMA', 'BT',
    'BUTNTAN', 'SIVONAC', 'I.B.', 'CRONAVAC', 'SINAVAC',
    'SINVAC', 'CORNONAVAC', 'BUT',
]

astrazeneca_list = [
    'ZENECA', 'OXFOR', 'ZENICA', 'FIOCRUZ', 'CRUZ', 'AZT',
    'CHADOX1', "INSTITUTO SERUM", 'COVISHILD', 'COVISHIELD',
    'ASTR', 'ATZ', 'FORD', 'OXF', '0XF', 'COVISHEID',
    'COVISCHELD', 'ATRAZENCA', 'FIOCROZ', 'OSWALDO', 'SHIELD',
    'INDIA', 'FIOCFRUZ', 'CRIZ', 'ABX0529', 'OXOFRD', 'FIO RUZ',
    'COVIDCHIELD', 'CORVISHELD', 'COREISHIEL', 'ATRAZANICA',
    'FIOCURZ', 'INSTITUTO SERIUM', 'AZ', 'BIOMANGUINHO',
    'FIO CRUOZ', 'CHADOX', 'COVISHIED', 'SERUM', 'FIOCRUS'
]

pfizer_list = [
    'FIZER', 'PFZES', 'PFIZAR', 'PIZER', 'IZER', 'BIONTECH',          
    'PZIFER', 'FAZER', 'EZER', 'PFI', 'PFZ', 'PFA', 'FAYZER',
    'PZF', 'PZI',  'BIOTECNO', 'BNT162B2', 'PAIFFER', 'BIOTECH',
    'COMIRNATY', 'PFYZER', 'BIO N TECH', 'PZHIER', 'FAISER' 
]

janssen_list = [
    'JANSSEN', 'JANSEN', 'UNICA', 'JAH', 'JANSE', 'JENS', 'JANHSEN',
    'JASSEN', 'JONHSON', 'JONSSON', 'JANSON', 'JHONSON', 'JOHNSON',
    'JOHSON', 'JHONNSONN', 'JONHOSON', 'JHANSSEN', 'JHANSEN',
    'JONSSEN', 'JHONSOM', 'JASEN', 'JANSSER',  'JHONSSEN',
    'JANSSEM', 'JONSHON', 'JANNSEN',
]

others = []

In [10]:
# 0 = astrazeneca, 1 = coronavac, 2 = pfizer, 3 = janssen
df = df.withColumn(
    'vac_number',
    F.when(
        (df.LAB_PR_COV).isin(astrazeneca_list),
        F.lit(0)
    ).when(
        (df.LAB_PR_COV).isin(coronavac_list),
        F.lit(1)
    ).when(
        (df.LAB_PR_COV).isin(pfizer_list),
        F.lit(2)
    ).when(
        (df.LAB_PR_COV).isin(janssen_list),
        F.lit(3)
    ).otherwise(F.lit(None))
)

In [11]:
df = df.withColumn(
    'DT_INTERNA',
    F.when(
        F.col('DT_INTERNA').isNull(),
        F.col('DT_ENTUTI')
    ).otherwise(F.col('DT_INTERNA'))
)

In [12]:
df = df.withColumn(
    'DT_ENTUTI',
    F.when(
        (F.col('DT_ENTUTI').isNull()) &
        (F.col('UTI') == 1),
        F.col('DT_INTERNA')
    ).otherwise(F.col('DT_ENTUTI'))
)

In [13]:
df = df.withColumn(
    'DT_SAIDUTI',
    F.when(
        (F.col('DT_SAIDUTI').isNull()) &
        (F.col('DT_ENTUTI').isNotNull()),
        F.col('DT_EVOLUCA')
    ).otherwise(F.col('DT_SAIDUTI'))
)

df = df.withColumn(
    'DT_SAIDUTI',
    F.when(
        (F.col('DT_SAIDUTI').isNull()) &
        (F.col('DT_ENTUTI').isNotNull()),
        F.col('DT_ENCERRA')
    ).otherwise(F.col('DT_SAIDUTI'))
)

In [14]:
df = df.withColumn(
    'DT_EVOLUCA',
    F.when(
        F.col('DT_EVOLUCA').isNull(),
        F.col('DT_ENCERRA')
    ).otherwise(F.col('DT_EVOLUCA'))
)

df = df.withColumn(
    'DT_EVOLUCA',
    F.when(
        F.col('DT_EVOLUCA').isNull(),
        F.col('DT_ENCERRA')
    ).otherwise(F.col('DT_SAIDUTI'))
)

In [15]:
data = [ col for col in df.columns if col.startswith('DT_') ]
data.append('DOSE_1_COV')
data.append('DOSE_2_COV')

In [16]:
for col in data:
    df = df.withColumn(
        col,
        F.when(
            F.col(col).isNotNull(), 
            F.to_date(col, 'dd/MM/yyyy')
        ).otherwise(F.col(col))
    )

In [17]:
df = df.withColumn(
    'date',
    F.when(
        F.col('DT_SIN_PRI').isNotNull(), 
        F.col('DT_SIN_PRI')
    ).when(
        F.col('DT_SIN_PRI').isNull(),
        F.when(
            F.col('DT_COLETA').isNotNull(),
            F.col('DT_COLETA')
        ).when(
            F.col('DT_COLETA').isNull(),
            F.col('DT_PCR')
        ).when(
            F.col('DT_NOTIFIC').isNotNull(),
            F.col('DT_NOTIFIC')
        )
    )
)

In [18]:
df = df.withColumn('newCases_SRAG', F.when(F.col('CLASSI_FIN') == 5, F.lit(1)))

In [19]:
df = df.withColumn(
    'new_des_SRAG',
    F.when(
        (F.col('CLASSI_FIN') == 1) |
        (F.col('CLASSI_FIN') == 2) |
        (F.col('CLASSI_FIN') == 3),
        F.lit(1)
    )
)

In [20]:
df = df.withColumn(
    'new_undefined_SRAG',
    F.when(
        (F.col('CLASSI_FIN') == 4) | 
        (F.col('CLASSI_FIN').isNull()),
        F.lit(1)
    )
)

In [21]:
df = df.withColumn(
    'newDeath_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('EVOLUCAO') == 2),
        F.lit(1)
    )
)

df = df.withColumn(
    'newDeath_des_SRAG',
    F.when(
        (F.col('new_des_SRAG').isNotNull()) &
        (F.col('EVOLUCAO') == 2),
        F.lit(1)
    )
)

df = df.withColumn(
    'newDeath_undefined_SRAG',
    F.when(
        (F.col('new_undefined_SRAG').isNotNull()) &
        (F.col('EVOLUCAO') == 2),
        F.lit(1)
    )
)

In [22]:
df = df.withColumn(
    'new_recovered_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('EVOLUCAO') == 1),
        F.lit(1)
    )
)

df = df.withColumn(
    'new_recovered_des_SRAG',
    F.when(
        (F.col('new_des_SRAG').isNotNull()) &
        (F.col('EVOLUCAO') == 1),
        F.lit(1)
    )
)

df = df.withColumn(
    'new_recovered_undefined_SRAG',
    F.when(
        (F.col('new_undefined_SRAG').isNotNull()) &
        (F.col('EVOLUCAO') == 1),
        F.lit(1)
    )
)

In [23]:
df = df.withColumn(
    'num_ind_vac_SRAG',
    F.when(
        F.col('VACINA_COV') == 1,
        F.lit(1)
    )
)

In [24]:
df = df.withColumn(
    'num_ind_incom_vac_SRAG',
    F.when(
        (F.col('DOSE_1_COV').isNotNull()) &
        (F.col('DOSE_2_COV').isNull()) &
        (F.col('num_ind_vac_SRAG').isNotNull()) &
        (F.col('vac_number') != 3),
        F.lit(1)
    )
)

In [25]:
df = df.withColumn(
    'num_ind_com_vac_SRAG',
    F.when(
        ((F.col('DOSE_1_COV').isNotNull()) &
        (F.col('DOSE_2_COV').isNotNull()) &
        (F.col('num_ind_vac_SRAG').isNotNull())) |
        (F.col('vac_number') == 3),
        F.lit(1)
    )
)

In [26]:
df = df.withColumn('comp', F.lit('2022-04-19'))

In [27]:
df = df.withColumn(
    'num_ind_incom_delay_less2_vac_SRAG',
    F.when(
        F.col('num_ind_incom_vac_SRAG').isNotNull(),
        F.when(
            F.months_between(F.col('comp'), F.col('DOSE_1_COV')) < 2,
            F.lit(1)
        )        
    )
)

df = df.withColumn(
    'num_ind_incom_delay_2_6_vac_SRAG',
    F.when(
        F.col('num_ind_incom_vac_SRAG').isNotNull(),
        F.when(
            (F.months_between(F.col('comp'), F.col('DOSE_1_COV')) >= 2) &
            (F.months_between(F.col('comp'), F.col('DOSE_1_COV')) < 6),
            F.lit(1)
        )        
    )
)

df = df.withColumn(
    'num_ind_incom_delay_more6_vac_SRAG',
    F.when(
        F.col('num_ind_incom_vac_SRAG').isNotNull(),
        F.when(
            F.months_between(F.col('comp'), F.col('DOSE_1_COV')) >= 6,
            F.lit(1)
        )        
    )
)

In [28]:
df = df.withColumn(
    'num_dose_sec_uni_delay_less2_vac_SRAG',
    F.when(
        F.col('num_ind_com_vac_SRAG').isNotNull(),
        F.when(
            F.col('DOSE_2_COV').isNotNull(),
            F.when(
                F.months_between(F.col('comp'), F.col('DOSE_2_COV')) < 2,
                F.lit(1)
            ).when(
                F.col('DOSE_2_COV').isNull(),
                F.when(
                    F.col('DOSE_1_COV').isNotNull(),
                    F.when(
                        F.months_between(F.col('comp'), F.col('DOSE_1_COV')) < 2,
                        F.lit(1)
                    )
                )
            )
        )
    )

)
df = df.withColumn(
    'num_dose_sec_uni_delay_2_6_vac_SRAG',
    F.when(
        F.col('num_ind_com_vac_SRAG').isNotNull(),
        F.when(
            F.col('DOSE_2_COV').isNotNull(),
            F.when(
                (F.months_between(F.col('comp'), F.col('DOSE_2_COV')) >= 2) &
                (F.months_between(F.col('comp'), F.col('DOSE_2_COV')) < 6),
                F.lit(1)
            ).when(
                F.col('DOSE_2_COV').isNull(),
                F.when(
                    F.col('DOSE_1_COV').isNotNull(),
                    F.when(
                        (F.months_between(F.col('comp'), F.col('DOSE_2_COV')) >= 2) &
                        (F.months_between(F.col('comp'), F.col('DOSE_2_COV')) < 6),
                        F.lit(1)
                    )
                )
            )
        )
    )
)

df = df.withColumn(
    'num_dose_sec_uni_delay_more6_vac_SRAG',
    F.when(
        F.col('num_ind_com_vac_SRAG').isNotNull(),
        F.when(
            F.col('DOSE_2_COV').isNotNull(),
            F.when(
                F.months_between(F.col('comp'), F.col('DOSE_2_COV')) >= 6,
                F.lit(1)
            ).when(
                F.col('DOSE_2_COV').isNull(),
                F.when(
                    F.col('DOSE_1_COV').isNotNull(),
                    F.when(
                        F.months_between(F.col('comp'), F.col('DOSE_1_COV')) >= 6,
                        F.lit(1)
                    )
                )
            )
        )
    )
)

In [29]:
df = df.withColumn(
    'ocu_ICU_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('UTI') == 1),
        F.lit(1)
    )
)

In [30]:
df = df.withColumn(
    'ocup_clinc_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('UTI') == 2),
        F.lit(1)
    )
)

In [31]:
df = df.withColumn(
    'newDeath_clinc_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('UTI') == 2) &
        (F.col('EVOLUCAO') == 2),
        F.lit(1)
    )
)

df = df.withColumn(
    'newDeath_UTI_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('UTI') == 1) &
        (F.col('EVOLUCAO') == 2),
        F.lit(1)
    )
)

In [32]:
df = df.withColumn(
    'newDeath_recovered_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('UTI') == 2) &
        (F.col('EVOLUCAO') == 1),
        F.lit(1)
    )
)

df = df.withColumn(
    'newDeath_recovered_UTI_SRAG',
    F.when(
        (F.col('newCases_SRAG').isNotNull()) &
        (F.col('UTI') == 1) &
        (F.col('EVOLUCAO') == 1),
        F.lit(1)
    )
)

In [33]:
df = df.withColumn(
    'CO_MUN_RES',
    F.when(
        F.col('CO_MUN_RES').isNull(),
        F.col('CO_MUN_NOT')
    ).otherwise(F.col('CO_MUN_RES'))
).withColumnRenamed('CO_MUN_RES', 'code_mun_ibge')

In [34]:
columns = [
    'newDeath_recovered_UTI_SRAG',
     'newDeath_recovered_SRAG',
     'newDeath_UTI_SRAG',
     'newDeath_clinc_SRAG',
     'ocup_clinc_SRAG',
     'ocu_ICU_SRAG',
     'num_dose_sec_uni_delay_more6_vac_SRAG',
     'num_dose_sec_uni_delay_2_6_vac_SRAG',
     'num_dose_sec_uni_delay_less2_vac_SRAG',
     'num_ind_incom_delay_more6_vac_SRAG',
     'num_ind_incom_delay_2_6_vac_SRAG',
     'num_ind_incom_delay_less2_vac_SRAG',
     'num_ind_com_vac_SRAG',
     'num_ind_incom_vac_SRAG',
     'num_ind_vac_SRAG',
     'new_recovered_undefined_SRAG',
     'new_recovered_des_SRAG',
     'new_recovered_SRAG',
     'newDeath_undefined_SRAG',
     'newDeath_des_SRAG',
     'newDeath_SRAG',
     'new_undefined_SRAG',
     'new_des_SRAG',
     'newCases_SRAG'
]

In [35]:
df.groupby('date', 'code_mun_ibge') \
.agg(
    *[F.count(c).alias(c) for c in columns],
    F.sum('ocu_ICU_SRAG').alias('num_ind_UTI_SRAG'),
    F.sum('ocup_clinc_SRAG').alias('num_ind_clin_SRAG')
).filter(F.col('date') >= '2020-01-01') \
.orderBy('date') \
.write.csv(os.path.join(datalake, version, 'preprocess', 'srag'), header=True, mode='overwrite')

22/04/25 17:35:35 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
